# Processing ssm info for ete visualization in Python

Note: markdown syntax can be reviewed here: https://daringfireball.net/projects/markdown/syntax#philosophy

- varGroups.rds files need to be converted into python-readable forms first
- varGroups and snv.out need to be matched
- Need to intersect snv information with GOI (pull out cancer drivers)
- unzip all mut assignments
- DFS tree in order to figure out ancestry and assign mutations
- [strike] Generate heatmap face attributes for each node in phylo tree
- add text annotation of protein symbol + aa change for nodes where they appear
- Visualize in ETE


***
#### Grab tsv files containing SNV information
!! NOTE: will also need to grab mutassignments at some point. Don't need snv.out though, since we already know the ssm ids

In [8]:
import glob

In [10]:
infiles = glob.glob('/Volumes/yjiao-1/phylowgs2/inputs/*.tsv')

#### Using Pandas to read the cnv file into a pandas DataFrame

In [154]:
from pandas import read_csv
i = 4
snvs = read_csv(infiles[i], delimiter='\t')

In [153]:
infiles

['/Volumes/yjiao-1/phylowgs2/inputs/208T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/253T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/51T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/272T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/39T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/42T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/122T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/98T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/25T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/155T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/PD004T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/148T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/62T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/131T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/99T_varGroups.tsv',
 '/Volumes/yjiao-1/phylowgs2/inputs/T33T_varGroups.tsv']

- - -
#### Some interesting methods for inspecting pandas dataframes

In [259]:
#snvs.info()
#snvs.dtypes
#snvs.describe()
#snvs.head()

# slicing
# snvs[['id', 'Hugo_Symbol']].head()

# logical indexing
# snvs[snvs.Hugo_Symbol=='B2M']

# set index
# Note THIS IS NOT IN PLACE BY DEFAULT!
snvs.set_index('id', inplace=True)
snvs.head()

# indexing should now be done w/ snvs.ix['s0']

# reset index back to default
#snvs.reset_index(inplace=True)

,Hugo_Symbol,Chromosome,Start_position,End_position,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Match_Norm_Seq_Allele1,...,q_hat,HS_q_hat_1,HS_q_hat_2,samp,order,ID,cnv_field,gene,a,d
id,,,,,,,,,,,,,,,,,,,,,
s0,ZRANB1,10,126631628,126631629,In_Frame_Ins,INS,-,-,GGACTG,NaN,...,2,1,1,SM-61EWU,1,g.chr10:126631628_126631629insGGACTG_1_1,"s0,1,1",ZRANB1,"264,152,92,33,207,194,274","264,152,92,58,207,194,274"
s1,PWWP2B,10,134218579,134218579,Missense_Mutation,SNP,C,C,T,NaN,...,2,1,1,SM-61EWU,1,g.chr10:134218579C>T_1_1,"s1,1,1",PWWP2B,"44,35,92,148,67,40,274","89,51,92,295,117,81,274"
s2,PWWP2B,10,134218579,134218579,Missense_Mutation,SNP,C,C,T,NaN,...,3,1,2,SM-61EWU,1,g.chr10:134218579C>T_1_2,"s2,1,2",PWWP2B,"264,152,86,377,207,194,77","264,152,257,377,207,194,230"
s3,TTC40,10,134738230,134738230,Missense_Mutation,SNP,C,C,T,NaN,...,2,1,1,SM-61EWU,1,g.chr10:134738230C>T_1_1,"s3,1,1",TTC40,"37,26,92,78,34,194,274","59,48,92,156,59,194,274"
s4,TTC40,10,134738230,134738230,Missense_Mutation,SNP,C,C,T,NaN,...,3,1,2,SM-61EWU,1,g.chr10:134738230C>T_1_2,"s4,1,2",TTC40,"264,152,21,377,207,194,65","264,152,64,377,207,194,140"


## Import GOI information

In [260]:
goi = read_csv('/Volumes/yjiao-1/phylowgs2/output/goi_from_moshe_20160222.csv', delimiter=',')
goi.head()

,Gene Name,Popular Name,Function,Location,Category,Pathway,Chr,Pos
0,PSMB8,LMP7,"Catalytic unit of the immunoproteosome, suited...",6p21.32,Genes involved in the processing of class I HL...,HLA,NaN,NaN
1,PSMB9,LMP2,"Catalytic unit of the immunoproteosome, suited...",6p21.32,Genes involved in the processing of class I HL...,HLA,NaN,NaN
2,PSMB10,LMP10 (MECL1),"Catalytic unit of the immunoproteosome, suited...",16q22.1,Genes involved in the processing of class I HL...,HLA,NaN,NaN
3,ERAP1,ERAP1,involved in trimming HLA class I-binding precu...,5q15,Peptide triming for generation of most HLA-I b...,HLA,NaN,NaN
4,ERAP2,ERAP2,Acts as a monomer or hetrodimer with ERAP1,5q15,Peptide triming for generation of most HLA-I b...,HLA,NaN,NaN


In [261]:
goi.Pathway = map(str.strip,goi.Pathway)
goi['Gene Name'] = map(str.strip,goi['Gene Name'])
drivers = goi[goi.Pathway=='Driver genes in melanoma']
drivers = set(drivers['Gene Name'])

***
## Intersect varGroup and GOI

Note that some of these might be the same genomic change, but with different copy numbers. We want to merge them under the assumption that each genomic change only happened once.

In [305]:
goiflag = [True if a in drivers else False for a in list(snvs.Hugo_Symbol) ]
dsnvs = snvs[goiflag]

In [301]:
import json
# temporary for now, later we'll know which one we actually want
patient = '39T'
tree = 2256
path = '/Volumes/yjiao-1/phylowgs2/output/witness/data/' + patient + '/mutass/' + str(tree) + '.json'
with open(path) as f:
    ms = f.readline()
ms = json.loads(ms)

In [323]:
for population in ms['mut_assignments']:
    for ssm in dsnvs.index:
        if ssm in ms['mut_assignments'][population]['ssms']:
            info = list(dsnvs.loc[ssm][['Hugo_Symbol', 'Protein_Change', 'HS_q_hat_1', 'HS_q_hat_2']])
            mutstring = info[0] + ' ' + info[1].replace('p.','') + ' copies: ' + ','.join(map(str,info[2:]))
            print mutstring

BRAF V600E copies: 1,2
GNA11 R183C copies: 2,2
BRAF V600E copies: 2,2


In [309]:
dsnvs.keys()

Index([u'Hugo_Symbol', u'Chromosome', u'Start_position', u'End_position',
       u'Variant_Classification', u'Variant_Type', u'Reference_Allele',
       u'Tumor_Seq_Allele1', u'Tumor_Seq_Allele2', u'Match_Norm_Seq_Allele1',
       u'Match_Norm_Seq_Allele2', u'Genome_Change', u'Codon_Change',
       u'ref_context', u'Annotation_Transcript', u'Protein_Change',
       u'Description', u'alt', u'ref', u'ccf_hat', u'ccf_CI95_low',
       u'ccf_CI95_high', u'detection_power', u'purity',
       u'detection_power_for_single_read', u'q_hat', u'HS_q_hat_1',
       u'HS_q_hat_2', u'samp', u'order', u'ID', u'cnv_field', u'gene', u'a',
       u'd'],
      dtype='object')

In [299]:
with open('/Volumes/yjiao-1/phylowgs2/inputs/39T.cnv') as f:
    test = read_csv(f, delimiter='\t')
for entry in test.ssms:
    try:
        ssms = entry.split(';')
        ssms = [s.split(',')[0] for s in ssms]
        if 's397' in ssms or 's398' in ssms:
            print test[test.ssms==entry]
    except:
        pass
    

      cnv                                                 a  \
732  c732  669124,669124,669124,334562,669124,669124,669124   

                                                    d  \
732  669124,669124,669124,669124,669124,669124,669124   

                           ssms  
732  s386,1,2;s393,1,2;s397,1,2  
      cnv                                                  a  \
733  c733  8980322,3607298,14677362,17960644,14677362,122...   

                                                     d  \
733  17960644,7214596,29354724,17960644,29354724,24...   

                                                  ssms  
733  s387,2,2;s388,2,2;s391,2,2;s392,2,2;s394,2,2;s...  


In [265]:
dsnvs.keys()

Index([u'Hugo_Symbol', u'Chromosome', u'Start_position', u'End_position',
       u'Variant_Classification', u'Variant_Type', u'Reference_Allele',
       u'Tumor_Seq_Allele1', u'Tumor_Seq_Allele2', u'Match_Norm_Seq_Allele1',
       u'Match_Norm_Seq_Allele2', u'Genome_Change', u'Codon_Change',
       u'ref_context', u'Annotation_Transcript', u'Protein_Change',
       u'Description', u'alt', u'ref', u'ccf_hat', u'ccf_CI95_low',
       u'ccf_CI95_high', u'detection_power', u'purity',
       u'detection_power_for_single_read', u'q_hat', u'HS_q_hat_1',
       u'HS_q_hat_2', u'samp', u'order', u'ID', u'cnv_field', u'gene', u'a',
       u'd'],
      dtype='object')

In [224]:
snvids = map(str, ms['mut_assignments']['1']['ssms'])
snvs.ix[snvids]

,Hugo_Symbol,Chromosome,Start_position,End_position,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Match_Norm_Seq_Allele1,...,q_hat,HS_q_hat_1,HS_q_hat_2,samp,order,ID,cnv_field,gene,a,d
id,,,,,,,,,,,,,,,,,,,,,
s206,RGS5,1,163138132,163138132,Missense_Mutation,SNP,C,C,T,NaN,...,4,2,2,SM-61EWU,1,g.chr1:163138132C>T_2_2,"s206,2,2",RGS5,"109,52,9,18,51,72,14","218,104,18,35,102,143,27"
s204,TMEM82,1,16073485,16073485,Missense_Mutation,SNP,C,C,T,NaN,...,8,2,6,SM-61EWU,1,g.chr1:16073485C>T_2_6,"s204,2,6",TMEM82,"142,152,148,380,124,94,266","570,152,593,1519,496,374,1065"
s202,RSC1A1,1,15988180,15988180,Missense_Mutation,SNP,T,T,C,NaN,...,8,2,6,SM-61EWU,1,g.chr1:15988180T>C_2_6,"s202,2,6",RSC1A1,"36,152,5,45,29,26,42","145,152,21,180,116,102,169"
s201,FCRL6,1,159779992,159779992,Missense_Mutation,SNP,C,C,T,NaN,...,4,2,2,SM-61EWU,1,g.chr1:159779992C>T_2_2,"s201,2,2",FCRL6,"62,28,10,130,60,32,62","124,56,20,261,120,64,125"
s200,DARC,1,159176200,159176200,Missense_Mutation,SNP,G,G,A,NaN,...,4,2,2,SM-61EWU,1,g.chr1:159176200G>A_2_2,"s200,2,2",DARC,"72,24,8,128,36,39,82","145,47,15,257,72,78,163"
s209,PAPPA2,1,176659467,176659467,Missense_Mutation,SNP,C,C,T,NaN,...,4,2,2,SM-61EWU,1,g.chr1:176659467C>T_2_2,"s209,2,2",PAPPA2,"155,97,17,348,157,118,217","310,194,34,696,314,235,434"
s208,TNR,1,175372542,175372542,Missense_Mutation,SNP,G,G,A,NaN,...,4,2,2,SM-61EWU,1,g.chr1:175372542G>A_2_2,"s208,2,2",TNR,"118,58,36,319,106,104,216","235,115,73,638,211,208,433"
s57,OR51I2,11,5475053,5475053,Missense_Mutation,SNP,C,C,T,NaN,...,4,2,2,SM-61EWU,1,g.chr11:5475053C>T_2_2,"s57,2,2",OR51I2,"126,75,19,166,110,85,274","251,150,38,332,221,170,274"
s55,LRP4,11,46916350,46916350,Missense_Mutation,SNP,A,A,G,NaN,...,4,2,2,SM-61EWU,1,g.chr11:46916350A>G_2_2,"s55,2,2",LRP4,"280,185,85,808,350,189,274","559,370,170,1617,700,378,274"


,Hugo_Symbol,Protein_Change
id,,
s163,GNA11,p.R183C
s397,BRAF,p.V600E
s398,BRAF,p.V600E
